# Calculate Pathweigh analysis, UDP, activity and consistency.

- Run all the following cells.
- Input file is should be named input.csv and can be either microarray or next gen RNA sequence data.

In [13]:
#Input file should be named input.csv (an example file is provided).
!rm -r Pathweigh
!git clone https://github.com/zurkin1/Pathweigh.git
!mv Pathweigh/*.* .

Cloning into 'Pathweigh'...
remote: Enumerating objects: 79, done.
remote: Counting objects: 100% (79/79), done.
remote: Compressing objects: 100% (73/73), done.
remote: Total 79 (delta 35), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (79/79), done.


## Calculate UDP

In [15]:
from udp import *
#Change to True in case of RNASeq data.
udp = calc_udp_multi_process(is_rnaseq = False)

Fri Jun 19 13:33:48 2020 Calculate UDP, is_rnaseq: False
............Fri Jun 19 13:34:21 2020 Done.


## Calculate Activity and Consistency

In [16]:
from activity import *
activity_obj = path_activity(udp, is_rnaseq = False)
activity = activity_obj.calc_activity_consistency_multi_process()

Fri Jun 19 13:34:26 2020 Init activity object
Fri Jun 19 13:34:26 2020 Calculate activity and consistency...
...Fri Jun 19 13:34:33 2020 Done.


## Select Path For Export

In [17]:
import ipywidgets as widgets
paths = pd.read_csv('pathologist.db.txt', delimiter='\t', header=None)
options = paths[0].unique()
dd = widgets.Dropdown(options=options, description='Select a path', disabled=False)
dd

Dropdown(description='Select a path', options=('granzyme a mediated apoptosis pathway(BioCarta)', 'hiv-1 nef: …

## Calculate and Download KGML file.
- Select the required sample number by changing parameter sample_num.

In [18]:
xml_result = activity_obj.xmlparser(path_id=dd.value, sample_num=0)
#from IPython.display import FileLink
#FileLink('output_path.xml')

from google.colab import files
files.download('output_path.xml')

Create Kegg XML for path: granzyme a mediated apoptosis pathway(BioCarta), sample: 0


##Draw a KGML map for path activity.

In [ ]:
activity_obj.graphparser(path_id=dd.value, sample_num=0)